In [19]:
from ray.rllib.offline.json_reader import JsonReader
import numpy as np
import tensorflow as tf

In [20]:
input_reader = JsonReader("logs2")

2022-12-16 09:45:49,107	WARNING json_reader.py:238 -- Treating input directory as glob patterns: ['/tf/Cage/Notebooks/logs2/*.json', '/tf/Cage/Notebooks/logs2/*.zip']


In [35]:
sequence_length=10
num_episodes = 1000
windows_per_episode = 100-sequence_length
state_len = 52
num_action = 145
encoding_len = state_len + num_action
state_to_index = {}
index_to_state = {}
state_index = 0

data_points = num_episodes * windows_per_episode

X = np.zeros((data_points,sequence_length,encoding_len))
Y_List = []

index = 0
for e in range(num_episodes):
    data = input_reader.next()
    data['obs'].shape
    for i in range(0, windows_per_episode):
        if not data['obs'][i+1].tobytes() in state_to_index:
            state_to_index[data['obs'][i+1].tobytes()] = state_index
            index_to_state[state_index] = data['obs'][i+1].tobytes()
            state_index += 1
        ns = state_to_index[data['obs'][i+1].tobytes()]
        if i < sequence_length:
            s = np.zeros((sequence_length, encoding_len))
            for k in range(0,i):
                s[k,:] = np.concatenate([data['obs'][k], np.zeros(num_action)])
        else:
            vec = np.zeros((sequence_length, num_action))
            vec[np.arange(sequence_length),data['actions'][i-sequence_length:i]] = 1
            s = np.concatenate([data['obs'][i-sequence_length:i], vec], axis=1)
            
        X[index,:,:] = s
        Y_List.append(ns)
        index += 1
index

90000

In [36]:
np.save('state_to_index.npy', state_to_index) 
np.save('index_to_state.npy', index_to_state)

In [41]:
Y = np.zeros((len(Y_List), state_index))

for i, y in enumerate(Y_List):
    Y[i,y] = 1

In [42]:
X[5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [43]:
Y[5]

array([0., 0., 0., ..., 0., 0., 0.])

In [44]:
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import Bidirectional


model = Sequential()
model.add(Bidirectional(LSTM(128, activation='relu', return_sequences=True), input_shape=(sequence_length, encoding_len)))
model.add(Bidirectional(LSTM(128, activation='relu', return_sequences=True)))
model.add(Flatten())
model.add(Dense(state_index, activation='softmax'))
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy())

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

print(model.summary())

model.fit(X, Y, epochs=100, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=128)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_4 (Bidirectio  (None, 10, 256)          333824    
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 10, 256)          394240    
 nal)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 2560)              0         
                                                                 
 dense_2 (Dense)             (None, 3535)              9053135   
                                                                 
Total params: 9,781,199
Trainable params: 9,781,199
Non-trainable params: 0
_________________________________________________________________
None
Train on 81000 samples, validate on 900

In [45]:
print(model.predict(np.array([X[10]]))[0].shape)

(3535,)


In [46]:
print(np.argmax(model.predict(np.array([X[10]]))))

3


In [47]:
print(np.argmax(Y[10]))

9


In [48]:
p = model.predict(np.array([X[10]]))[0]
p[np.where(Y[10]==1)]

array([0.03573096], dtype=float32)

In [49]:
p.shape

(3535,)

In [50]:
np.save('target.npy',X[10])
np.save('prediction.npy',p)